In [1]:
import pandas_ta as pta
import pandas as pd
import numpy as np

from typing import NamedTuple

from quantfreedom.enums import CandleProcessingType
from quantfreedom.exchanges.mufex import Mufex
from quantfreedom.strategies.strategy import Strategy
exchange = Mufex()
candles = exchange.get_candles()

%load_ext autoreload
%autoreload 2

since_date_ms=2023-09-28 04:57:04, until_date_ms=2023-09-28 21:37:04
Got 200 new candles for since=2023-09-28 04:57:04
Getting candles since=2023-09-28 21:35:01
Got 0 candles
It took 0.02 minutes to get the candles


In [2]:
rsi_strat = Strategy(candle_processing_mode=CandleProcessingType.BacktestCandleByCandle, candles=candles, num_candles=50)

In [8]:
for indicator_setting_index in range(rsi_strat.indicator_settings_array[0].shape[0]):
    rsi_strat.ind_settings_or_results(indicator_setting_index=indicator_setting_index)
    for bar_index in range(candles.shape[0]):
        rsi_strat.set_price_data(bar_index=bar_index)
        if rsi_strat.evaluate():
            print(
                f"Current RSI is {rsi_strat.rsi[bar_index]} For ind setting {indicator_setting_index} and bar num {bar_index} ... Its entry time baby"
            )



RSI lenth = 10 and RSI is below 30
Current RSI is 23.46 For ind setting 0 and bar num 165 ... Its entry time baby
Current RSI is 29.75 For ind setting 0 and bar num 166 ... Its entry time baby
Current RSI is 29.14 For ind setting 0 and bar num 196 ... Its entry time baby


RSI lenth = 10 and RSI is below 40
Current RSI is 39.55 For ind setting 1 and bar num 38 ... Its entry time baby
Current RSI is 36.36 For ind setting 1 and bar num 39 ... Its entry time baby
Current RSI is 36.52 For ind setting 1 and bar num 40 ... Its entry time baby
Current RSI is 39.97 For ind setting 1 and bar num 41 ... Its entry time baby
Current RSI is 38.33 For ind setting 1 and bar num 42 ... Its entry time baby
Current RSI is 39.1 For ind setting 1 and bar num 47 ... Its entry time baby
Current RSI is 35.76 For ind setting 1 and bar num 48 ... Its entry time baby
Current RSI is 38.57 For ind setting 1 and bar num 53 ... Its entry time baby
Current RSI is 38.65 For ind setting 1 and bar num 55 ... Its entr

In [ ]:
pd.DataFrame(rsi_strat.rsi).plot()